In [1027]:
import pandas as pd
import numpy as np
import seaborn as sns

In [1028]:
df=pd.read_csv('train.csv')

In [1029]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


In [1030]:
df['model'].value_counts()

F-150 XLT             780
M3 Base               682
X5 xDrive35i          615
F-150 Lariat          496
Mustang GT Premium    466
                     ... 
Suburban 2500           1
TLX Tech                1
Rogue Sport S           1
A8 4.0                  1
S-10 LS                 1
Name: model, Length: 1827, dtype: int64

In [1031]:
df.describe()

,id,model_year,milage,price
count,54273.000000,54273.000000,54273.000000,5.427300e+04
mean,27136.000000,2015.091979,72746.175667,3.921844e+04
std,15667.409917,5.588909,50469.490448,7.282634e+04
min,0.000000,1974.000000,100.000000,2.000000e+03
25%,13568.000000,2012.000000,32268.000000,1.550000e+04
50%,27136.000000,2016.000000,66107.000000,2.800000e+04
75%,40704.000000,2019.000000,102000.000000,4.500000e+04
max,54272.000000,2024.000000,405000.000000,2.954083e+06


In [1032]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54273 entries, 0 to 54272
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            54273 non-null  int64 
 1   brand         54273 non-null  object
 2   model         54273 non-null  object
 3   model_year    54273 non-null  int64 
 4   milage        54273 non-null  int64 
 5   fuel_type     54273 non-null  object
 6   engine        54273 non-null  object
 7   transmission  54273 non-null  object
 8   ext_col       54273 non-null  object
 9   int_col       54273 non-null  object
 10  accident      54273 non-null  object
 11  clean_title   54273 non-null  object
 12  price         54273 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 5.4+ MB


In [1033]:
df['horsepower']=df['engine'].apply(lambda x:float(x[:4]) if x[5:7]=='HP' else 331.687)

In [1034]:
df

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horsepower
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000,375.000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250,300.000
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000,300.000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500,335.000
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850,200.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54268,54268,BMW,X6 xDrive50i,2017,29000,Gasoline,445.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Brown,None reported,Yes,29000,445.000
54269,54269,Audi,A4 2.0T Premium,2015,94634,E85 Flex Fuel,220.0HP 2.0L 4 Cylinder Engine Flex Fuel Capab...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,6500,220.000
54270,54270,Porsche,Cayenne S,2013,40989,Gasoline,420.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Black,At least 1 accident or damage reported,Yes,18950,420.000
54271,54271,Porsche,911 Carrera 4 GTS,2023,1518,Gasoline,4.0L H6 24V GDI DOHC,8-Speed Automatic with Auto-Shift,Beige,Brown,None reported,Yes,194965,331.687


In [1035]:
import re
v1=df['engine'].str.extract(r'\s(\d+)\s')

In [1036]:
v1=pd.DataFrame({'index':range(0,54273),'number':v1[0]})
# v1[0]

In [1037]:
v2=df['engine'].str.extract(r'\s\w(\d+)\s')

In [1038]:
v2=pd.DataFrame({'index':range(0,54273),'number':v2[0]})

In [1039]:
newv=pd.merge(v1, v2, on='index', how='outer')

In [1040]:
newv.head()

,index,number_x,number_y
0,0,NaN,6
1,1,6,NaN
2,2,8,NaN
3,3,6,NaN
4,4,NaN,6


In [1041]:
df['v']=newv['number_x'].combine_first(newv['number_y'])

In [1042]:
df['v']=df['v'].fillna('6')
df['v'] = df['v'].astype(int)

In [1043]:
l1=df['engine'].str.extract(r'\s(\d+\.\d+)\w\s')
l1=pd.DataFrame({'index':range(0,54273),'number':l1[0]})

In [1044]:
l2=df['engine'].str.extract(r'\b(\d+\.\d+)\sLiter\b')
l2=pd.DataFrame({'index':range(0,54273),'number':l2[0]})

In [1045]:
newv=pd.merge(l1, l2, on='index', how='outer')
df['L']=newv['number_x'].combine_first(newv['number_y'])


In [1046]:
df['L']=df['L'].fillna('3.72')
df['L'] = df['L'].astype(float)

In [1047]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horsepower,v,L
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000,375.0,6,3.5
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250,300.0,6,3.0
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000,300.0,8,4.2
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500,335.0,6,3.0
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850,200.0,6,3.8


In [1048]:
df['transmission'].value_counts()

A/T                                                   16757
8-Speed A/T                                            7287
Transmission w/Dual Shift Mode                         6454
6-Speed A/T                                            5944
6-Speed M/T                                            3618
7-Speed A/T                                            3415
10-Speed A/T                                           2187
8-Speed Automatic                                      1189
5-Speed A/T                                            1185
9-Speed A/T                                            1073
4-Speed A/T                                             764
5-Speed M/T                                             718
CVT Transmission                                        692
10-Speed Automatic                                      642
6-Speed Automatic                                       516
Automatic                                               441
M/T                                     

In [1049]:
df['tranmission_no']=df['transmission'].str.extract(r'(\d+)')[0]

In [1050]:
df['tranmission_no']=df['tranmission_no'].fillna(0)

In [1051]:
df['transmission']=df['transmission'].astype('category').cat.codes

In [1052]:
acci=pd.get_dummies(df['accident'])
df=pd.concat([df,acci],axis=1)

In [1053]:
# int_col=pd.get_dummies(df['int_col'])
# df=pd.concat([df,int_col],axis=1)
df['int_col']=df['int_col'].astype('category').cat.codes

In [1054]:
clean_title=pd.get_dummies(df['clean_title'])
df=pd.concat([df,clean_title],axis=1)

In [1055]:
# ext_col=pd.get_dummies(df['ext_col'])
# df=pd.concat([df,ext_col],axis=1)
df['int_col']=df['int_col'].astype('category').cat.codes

In [1056]:
fuel_type=pd.get_dummies(df['fuel_type'])
df=pd.concat([df,fuel_type],axis=1)

In [1057]:
brand=pd.get_dummies(df['brand'])
df=pd.concat([df,brand],axis=1)

In [1058]:
df['model']=df['model'].astype('category').cat.codes

In [1059]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,...,Rivian,Rolls-Royce,Saturn,Scion,Subaru,Suzuki,Tesla,Toyota,Volkswagen,Volvo
0,0,Ford,644,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,2,Blue,57,...,0,0,0,0,0,0,0,0,0,0
1,1,BMW,49,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,18,Black,9,...,0,0,0,0,0,0,0,0,0,0
2,2,Jaguar,1771,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,14,Purple,6,...,0,0,0,0,0,0,0,0,0,0
3,3,BMW,1748,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,43,Gray,24,...,0,0,0,0,0,0,0,0,0,0
4,4,Pontiac,693,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,34,White,9,...,0,0,0,0,0,0,0,0,0,0


In [1060]:
df.drop(columns=['brand','fuel_type','ext_col','int_col','accident','clean_title','id'])

,model,model_year,milage,engine,transmission,price,horsepower,v,L,tranmission_no,...,Rivian,Rolls-Royce,Saturn,Scion,Subaru,Suzuki,Tesla,Toyota,Volkswagen,Volvo
0,644,2018,74349,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,2,11000,375.000,6,3.50,10,...,0,0,0,0,0,0,0,0,0,0
1,49,2007,80000,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,18,8250,300.000,6,3.00,6,...,0,0,0,0,0,0,0,0,0,0
2,1771,2009,91491,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,14,15000,300.000,8,4.20,6,...,0,0,0,0,0,0,0,0,0,0
3,1748,2022,2437,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,43,63500,335.000,6,3.00,0,...,0,0,0,0,0,0,0,0,0,0
4,693,2001,111000,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,34,7850,200.000,6,3.80,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54268,1745,2017,29000,445.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,27,29000,445.000,8,4.40,8,...,0,0,0,0,0,0,0,0,0,0
54269,165,2015,94634,220.0HP 2.0L 4 Cylinder Engine Flex Fuel Capab...,14,6500,220.000,4,2.00,6,...,0,0,0,0,0,0,0,0,0,0
54270,421,2013,40989,420.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,43,18950,420.000,6,3.60,0,...,0,0,0,0,0,0,0,0,0,0
54271,142,2023,1518,4.0L H6 24V GDI DOHC,29,194965,331.687,6,3.72,8,...,0,0,0,0,0,0,0,0,0,0


In [1061]:
arr=df['engine'].apply(lambda x:sorted(re.findall(r'\b[a-zA-Z]+\b',x)))
# arr[0]
colset=[]
for i in range(54273):
    for j in arr[i]:
        colset.append(j)

In [1062]:
enginedata=['AH',
 'Battery',
 'Capability',
 'Cylinder',
 'DDI',
 'DOHC',
 'Diesel',
 'Electric',
 'Engine',
 'Flat',
 'Flex',
 'Flexible',
 'Fuel',
 'GDI',
 'GTDI',
 'Gas',
 'Gasoline',
 'Hybrid',
 'Hydrogen',
 'In',
 'Intercooled',
 'Liter',
 'MPFI',
 'Mild',
 'Motor',
 'OHV',
 'PDI',
 'Plug',
 'Range',
 'Rotary',
 'SC',
 'SOHC',
 'Standard',
 'Straight',
 'Supercharged',
 'System',
 'TFSI',
 'TSI',
 'Turbo',
 'Twin',
 'ULEV',
 'VTEC',
 'engine']

In [1063]:
for i in enginedata:
    df[i]=0
for i in range(54273):
    for j in arr[i]:
        if j in enginedata:
            df.loc[i,j]=1

In [1064]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,...,Standard,Straight,Supercharged,System,TFSI,TSI,Turbo,Twin,ULEV,VTEC
0,0,Ford,644,2018,74349,Gasoline,0,2,Blue,57,...,0,0,0,0,0,0,0,0,0,0
1,1,BMW,49,2007,80000,Gasoline,0,18,Black,9,...,0,1,0,0,0,0,0,0,0,0
2,2,Jaguar,1771,2009,91491,Gasoline,0,14,Purple,6,...,0,0,0,0,0,0,0,0,0,0
3,3,BMW,1748,2022,2437,Hybrid,0,43,Gray,24,...,0,1,0,0,0,0,0,0,0,0
4,4,Pontiac,693,2001,111000,Gasoline,0,34,White,9,...,0,0,0,0,0,0,0,0,0,0


In [1065]:
df=df.dropna(axis=1)

In [1066]:
df=df.drop(columns=['brand','fuel_type','ext_col','int_col','accident','clean_title','id'])

In [1067]:
df.head()

,model,model_year,milage,engine,transmission,price,horsepower,v,L,tranmission_no,...,Standard,Straight,Supercharged,System,TFSI,TSI,Turbo,Twin,ULEV,VTEC
0,644,2018,74349,0,2,11000,375.0,6,3.5,10,...,0,0,0,0,0,0,0,0,0,0
1,49,2007,80000,0,18,8250,300.0,6,3.0,6,...,0,1,0,0,0,0,0,0,0,0
2,1771,2009,91491,0,14,15000,300.0,8,4.2,6,...,0,0,0,0,0,0,0,0,0,0
3,1748,2022,2437,0,43,63500,335.0,6,3.0,0,...,0,1,0,0,0,0,0,0,0,0
4,693,2001,111000,0,34,7850,200.0,6,3.8,0,...,0,0,0,0,0,0,0,0,0,0


In [1068]:
target=df['price']

In [1069]:
df=df.drop(columns=['price'])

In [1070]:
!pip3 install catboost

In [1071]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()
stanscaler=StandardScaler()
# dft=stanscaler.fit_transform(df)
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.00001, random_state=42)

In [1074]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostRegressor
catboost=CatBoostRegressor(loss_function='RMSE',depth=5,random_seed=42,iterations=3500,learning_rate=0.01,verbose=False)
# catboost=RandomForestClassifier( n_estimators=1000000, random_state=42)
catboost.fit(X_train, y_train)

In [1075]:
from sklearn.metrics import mean_squared_error
y_pred = catboost.predict(X_test)
rmse1 = np.sqrt(mean_squared_error(y_test, y_pred))
print("The Root mean square is " + str(rmse1))

The Root mean square is 11209.282380288118


In [1076]:
df1=pd.read_csv('test.csv')
df1.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,54273,Mercedes-Benz,E-Class E 350,2014,73000,Gasoline,302.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes
1,54274,Lexus,RX 350 Base,2015,128032,Gasoline,275.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Silver,Black,None reported,Yes
2,54275,Mercedes-Benz,C-Class C 300,2015,51983,Gasoline,241.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Blue,White,None reported,Yes
3,54276,Land,Rover Range Rover 5.0L Supercharged Autobiogra...,2018,29500,Gasoline,518.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,White,At least 1 accident or damage reported,Yes
4,54277,BMW,X6 xDrive40i,2020,90000,Gasoline,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes


In [1077]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36183 entries, 0 to 36182
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            36183 non-null  int64 
 1   brand         36183 non-null  object
 2   model         36183 non-null  object
 3   model_year    36183 non-null  int64 
 4   milage        36183 non-null  int64 
 5   fuel_type     36183 non-null  object
 6   engine        36183 non-null  object
 7   transmission  36183 non-null  object
 8   ext_col       36183 non-null  object
 9   int_col       36183 non-null  object
 10  accident      36183 non-null  object
 11  clean_title   36183 non-null  object
dtypes: int64(3), object(9)
memory usage: 3.3+ MB


In [1078]:
df1.replace('-','idontknow',inplace=True)

In [1079]:
df1['horsepower']=df1['engine'].apply(lambda x:float(x[:4]) if x[5:7]=='HP' else 306.838)

In [1080]:
v1=df1['engine'].str.extract(r'\s(\d+)\s')
v1=pd.DataFrame({'index':range(0,36183),'number':v1[0]})
v2=df1['engine'].str.extract(r'\s\w(\d+)\s')
v2=pd.DataFrame({'index':range(0,36183),'number':v2[0]})
newv=pd.merge(v1, v2, on='index', how='outer')
newv.head()
df1['v']=newv['number_x'].combine_first(newv['number_y'])
df1['v']=df1['v'].fillna('6')
df1['v'] = df1['v'].astype(int)
l1=df1['engine'].str.extract(r'\s(\d+\.\d+)\w\s')
l1=pd.DataFrame({'index':range(0,36183),'number':l1[0]})
l2=df1['engine'].str.extract(r'\b(\d+\.\d+)\sLiter\b')
l2=pd.DataFrame({'index':range(0,36183),'number':l2[0]})
newv=pd.merge(l1, l2, on='index', how='outer')
df1['L']=newv['number_x'].combine_first(newv['number_y'])
df1['L']=df1['L'].fillna('3.72')
df1['L'] = df1['L'].astype(float)

In [1081]:
df1['tranmission_no']=df1['transmission'].str.extract(r'(\d+)')[0]

In [1082]:
df1['tranmission_no']=df1['tranmission_no'].fillna(0)

In [1083]:
df1['transmission']=df1['transmission'].astype('category').cat.codes

In [1084]:
brand=pd.get_dummies(df1['brand'])
df1=pd.concat([df1,brand],axis=1)
fuel_type=pd.get_dummies(df1['fuel_type'])
df1=pd.concat([df1,fuel_type],axis=1)
# ext_col=pd.get_dummies(df1['ext_col'])
# df1=pd.concat([df1,ext_col],axis=1)
clean_title=pd.get_dummies(df1['clean_title'])
d1f=pd.concat([df1,clean_title],axis=1)
# int_col=pd.get_dummies(df1['int_col'])
# df1=pd.concat([df1,int_col],axis=1)
df1['int_col']=df1['int_col'].astype('category').cat.codes
df1['ext_col']=df1['ext_col'].astype('category').cat.codes
acci=pd.get_dummies(df1['accident'])
df1=pd.concat([df1,acci],axis=1)

In [1085]:
df1['model']=df1['model'].astype('category').cat.codes

In [1086]:
df1=df1.drop(columns=['brand','fuel_type','ext_col','int_col','accident','clean_title','id'])

In [1087]:
arr=df1['engine'].apply(lambda x:sorted(re.findall(r'\b[a-zA-Z]+\b',x)))
# arr[0]
colset=[]
for i in range(36183):
    for j in arr[i]:
        colset.append(j)

In [1088]:
for i in enginedata:
    df1[i]=0
for i in range(36183):
    for j in arr[i]:
        if j in enginedata:
            df1.loc[i,j]=1

In [1089]:
df1['Cylinder'][1]

1

In [1090]:
df1=df1.dropna(axis=1)

In [1091]:
# catboost.predict(df1)
df1.shape
df1['Yes']=1
df1['Maybach']=0
df1['Plymouth']=0
df1=df1.drop(['Saab', 'smart'],axis=1)

In [1092]:
set(df.columns.tolist())-set(df1.columns.tolist())
set(df1.columns.tolist())-set(df.columns.tolist())


set()

In [1093]:
result=pd.DataFrame({'id':pd.read_csv('test.csv')['id'],'price':catboost.predict(df1)})

In [1094]:
result.to_csv('out.csv',index=False)

In [1095]:
catboost.predict(df1)

array([23164.14652144, 21783.05535519, 28502.16490518, ...,
       12791.05023179, 56222.24053864, 18263.19299962])